In [1]:
import numpy as np
from poisson_id_link_model import Problem, Sim, fit

## Overview

We're modelling particle counts at $k$ detectors, caused by an event in a space X, discretized as $X^m$.

We assume that we can model a count at a detector $y_i$ as a Poisson distribution dependent on an event vector $x^m$,

$$
y_i \sim Poisson(\lambda_i)
$$

$$
\lambda_i = \alpha_i \cdot x^m + \beta_i = \theta_i \cdot x^m
$$

where:
* the $x^m$ vectors are sparse; they correspond to a point event
* the structure of $\theta_i$ is not clear

For example, say we've got 3 detectors, and we've discretized the space where decomposition events happen into $m=4$ cells.

The Poisson means $\lambda_i$ are given by

$$
\left[
\begin{matrix} 
\lambda_0 \\
\lambda_1 \\
\lambda_2 \\
\end{matrix}
\right]
=
\left[
\begin{matrix} 
\alpha_{00} & \alpha_{01} & \alpha_{02} & \alpha_{03} & \beta_0 \\
\alpha_{10} & \alpha_{11} & \alpha_{12} & \alpha_{13} & \beta_1 \\
\alpha_{20} & \alpha_{21} & \alpha_{22} & \alpha_{23} & \beta_2 \\
\end{matrix}
\right]
\cdot
\left[
\begin{matrix} 
x_0 \\
x_1 \\
x_2 \\
x_3 \\
1 \\
\end{matrix}
\right]
$$


This seems intuitive to me; an event in a given cell causes Poisson-distributed counts in a given detector; the dark count rate is taken care of by $\beta$; rates are linear in the events.

So we have 2 problems.

### Estimate the parameter vector $\theta$ given training data $(x, y)$

We have training tuples (presumably from the geant4 simulation) and we use them to estimate the $\theta$ coefficients.

### Given $\theta$ and detector responses $y$, find the most plausible event vector $x^m$

As I understand it, in each time slice we're assuming either 0 or 1 decomposition events happen.

If our event space has been discretized into $m$ cells, we have $m + 1$ alternative hypotheses $H = \{\overrightarrow{0}, \overrightarrow{e_1}, \overrightarrow{e_2}, \ldots, \overrightarrow{e_m}\}$ for each time slice that can explain the detector measurements: the unit vectors in $R^m$ and the $0^m$ vector.

So if we have a function proportional to the likelihood $l(y \mid \theta, x^m)$ implied by our Poisson model, we're looking for

$$
\underset{x^m \in H}{\text{argmax}} \, l(y \mid \theta, x^m)
$$

We get an easy $l$ function if we assume indepedence between detectors and simply multiply their likelihoods together.

This is where the l1 penalty might come in useful as the $x^m$ vector is very sparse.


### Problem characterization

Our problems are characterized by:
* the number of detectors $y\_dim$
* the number of discretized space voxels $x\_dim$

so let's come up with a dummy problem and try to recover $\theta$.

In [2]:
theta = np.array([
    [1.0, 0.5, 0.1, 0.1, 0.01],
    [0.5, 0.1, 1.0, 0.1, 0.01],
    [0.1, 1.0, 0.5, 0.1, 0.3], # this one's dcr is high
])
sim = Sim(theta)
xs, ys, eys = [], [], []
for i in range(100000):
    x, y = sim.sample()
    e_y = sim.obj.Ey_given_x(sim.theta, x)
    ys.append(y)
    eys.append(e_y)
    
# quick sanity check
print(np.mean(ys, axis = 0) - np.mean(eys, axis = 0))


[-0.002405 -0.001151 -0.001148]


Proof of concept solver:
* I'm being really braindead about it, not taking advantage of the sparsity of $x$ to get a quadratic speed up and give the solver an easier time
* I'm using the scipy [trust-constr solver](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-trustconstr.html) because we have a nonnegativity constraint
* I'm also using [autograd](https://github.com/HIPS/autograd) because it's super easy to get the gradient and I can use the built-in hessian vector product to give the solver second-order information (not that I know if it matters)

There must be a specialized solver for this somewhere, maybe in [cvxopt](https://cvxopt.org/) or in [The Bible](https://web.stanford.edu/~boyd/cvxbook/), but I try not to look at the Boyd & Vandenberghe book for too long because it makes my brain hurt.

In [3]:
samples = [sim.sample() for _ in range(100)]
theta_100, _sol = fit(sim.x_dim, sim.y_dim, samples)
frobenius_100 = np.linalg.norm(theta - theta_100, ord="fro")

Let's give it more samples and see if the frobenius norm of $\theta - \hat{\theta}$ goes down as expected as the number of samples goes up:

In [4]:
samples.extend([sim.sample() for _ in range(900)])
theta_1000, _sol = fit(sim.x_dim, sim.y_dim, samples)
frobenius_1000 = np.linalg.norm(theta - theta_1000, ord="fro")
print('frobenius 100: {0} 1000: {1}'.format(frobenius_100, frobenius_1000))

frobenius 100: 0.6413563129043409 1000: 0.16760840702314156
